In [1]:
from cral.pipeline.semantic_segmentation_pipeline import SemanticSegPipe
from cral.tracking import set_experiment

set_experiment('bb4b63b8-6457-42e1-b48f-d4d50d247d3b')

new_pipe = SemanticSegPipe()

# add data and create tfrecords

In [10]:
new_pipe.add_data(
    train_images_dir='./seg_data/mini_ADE20K/images/',
    train_anno_dir='./seg_data/mini_ADE20K/annotations/',
    annotation_format='grayscale',
    split=0.2)

new_pipe.lock_data()

# set up Deeplabv3+

In [9]:
from cral.models.semantic_segmentation.deeplabv3 import Deeplabv3Config
from cral.models.semantic_segmentation.Unet import UNetConfig
from cral.models.semantic_segmentation.PspNet import PspNetConfig

deeplab_config = Deeplabv3Config(
    height=576, 
    width=576, output_stride = 8)

new_pipe.set_algo(
    feature_extractor='resnet50',
    config=deeplab_config)

In [8]:
deeplab_config

# train

In [5]:
# print(new_pipe.model.summary())

new_pipe.train(
    num_epochs=150,
    snapshot_prefix='deeplab_resnet50',
    snapshot_path='/tmp/aniketjoshi/',
    snapshot_every_n=5
    )

# predict

In [6]:
from cral.models.semantic_segmentation import annotate_image
from cral.pipeline.semantic_segmentation_pipeline import SemanticSegPipe
from IPython.display import display
from cral.tracking import set_experiment

# set_experiment('487f0813-c080-4d8b-8a9d-792b0acf8ad9')

new_pipe = SemanticSegPipe()
new_pipe.dcrf = True #enable or disable dcrf

pred_func = new_pipe.prediction_model('./deeplab_resnet50/')

In [3]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [24]:
image_path = './seg_data/mini_ADE20K/images/ADE_val_00000011.jpg'

mask_array = pred_func(image_path)
segmented_image = annotate_image(image_path, mask_array)

display(segmented_image)